# Scope of work

1) Import all necessary libraries and modules.  
2) First we need to obtain inforamtion about all available cars from the first main pages.  
3) Create a SQL database and export there gathered info about cars from the main pages.  
4) Then we will call module dealers_cars to acquire links to every dealer's list of cars from all main pages.  
5) Now we will repeat the same procedure as in the 1st step but to the every dealer's cars list.  
6) Add extracted data to a created SQL database

## 1. Imort of libraries and modules

In [1]:
import main_pages
import parsing
import marks
import dataframe
import sql_db
import cars_scraper
from datetime import datetime

## 2. Acquring cars' information from the first main pages

Here we call the 1st module 'parsing' to scrap data from the main pages of the website. Then we store all gathered information about cars into corresponding lists.

In [2]:
# Link to the main webpage
url = 'https://www.autoscout24.com/lst?atype=C&desc=0&sort=standard&source=homepage_search-mask&ustate=N%2CU'

The module **main_pages** collects all URLs of the main pages from the website autoscout24.com

In [3]:
all_pages = main_pages.pages_urls(url)

With a help of **parsing** module we scrap info about cars from main pages (20).

In [4]:
start = datetime.now()
cars, characteristics, prices, locations = parsing.cars_info(all_pages)
end = datetime.now()
print('Total time :', end-start)

Total time : 0:00:26.102254


Here we call the module **marks** in order to extract all existing car marks from the website. Afterwards we will replace
spaces in marks' names into dashes '-'

In [5]:
marks_menu = marks.all_marks(url)

Here we call the module **dataframe** in order to gather all info about cars into one dataframe

In [6]:
df = dataframe.df_construct(marks_menu, cars, characteristics, prices, locations)

In [7]:
df

,mark,model,mileage,transmission,registration,fuel,power,location,price
0,Aston-Martin,DBX,0,Automatic,None,Gasoline,707,DE,282360
1,Mazda,MX-5,178290,Manual,08/2001,Gasoline,110,BE,4950
2,DS-Automobiles,DS 4,190000,Manual,02/2012,Diesel,111,IT,4990
3,BMW,520,257000,Manual,11/2013,Diesel,163,BE,8900
4,Porsche,Cayenne,164335,Automatic,02/2005,Gasoline,250,BE,9490
...,...,...,...,...,...,...,...,...,...
395,Volkswagen,Golf,60996,Automatic,07/2018,Gasoline,310,DE,29890
396,Opel,Insignia,59990,Automatic,12/2011,Gasoline,220,DE,9950
397,BMW,X4,73550,Automatic,05/2018,Diesel,163,BE,34900
398,Audi,A6,148799,Automatic,10/2015,Diesel,218,DE,22990


## 3. Creating a SQL database and exporting parsed data there from the main pages

Here we connect to another module **sql_db**. This module connects to a PostgreSQL database *autoscout*. In this database there is a schema *autoscout* which contains the main table *cars*.

In [9]:
sql_db.connect(df,'replace')

## 4. Acquring cars' information from the all the car dealers and storing into a SQL database

In [10]:
start = datetime.now()

for page in all_pages:
    cars_scraper.parser(page, marks_menu)

end = datetime.now()
print('Total time :', end-start)

Total time : 1:39:19.790364


Total time :1:28:41.573803